In [12]:
import pandas as pd
import numpy as np
import json
import requests

In [39]:
# !pip install folium

In [40]:
api_key = 'dcf8c004ab92b095154a8edbc67cae15'

In [41]:
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

In [66]:
# addr_to_lat_lon('울릉도')  #울릉도 없음

# 한국 위도 경도

In [64]:
# 한국 위도 경도
korea_city = ['강릉', '춘천', '광주', '대구', '대전', '부산', '서울', '전주', '제주', '청주']
korea = dict()
lst = []
addr_to_lat_lon('강릉')

# loc =str(korea_city[0])
# loc

for city in korea_city:
    loc = addr_to_lat_lon(city)
    lst.append(loc)
    korea['korea']=lst
korea

{'korea': [(128.875906144832, 37.7521080804166),
  (127.730207024668, 37.8812440340088),
  (127.255138719721, 37.4293941934866),
  (128.601805491072, 35.8713802646197),
  (127.38483484675, 36.3505388992836),
  (129.075087492149, 35.1798200522868),
  (126.978652258309, 37.566826004661),
  (127.148143800621, 35.8240731508775),
  (126.498229141199, 33.4889179032603),
  (127.489020156402, 36.6424871337289)]}

# 주변국 위도경도

In [6]:
# 찾을 도시들

areas = [ '혜산', '강계', '신의주', '함흥', '평양', '청진',
         '가고시마', '나고야', '난징', '다카마쓰','도쿄', '삿포로', '센다이', '오사카','후쿠오카', '히로시마'
        '베이징','상하이', '선양','창춘', '지난',
         '블라디보스톡'
        ]

# areas1 = ['선봉', '삼지연', '청진', '중강', '혜산', '강계', '풍산', '김책', '수풍', '장진', '신의주',
#        '구성', '희천', '함흥', '신포', '안주', '양덕', '원산', '평양', '남포', '장전', '사리원',
#        '신계', '용연', '해주', '개성', '평강', '가고시마', '나고야', '난징', '다롄', '다카마쓰',
#        '도쿄', '베이징', '블라디보스톡', '삿포로', '상하이', '선양', '센다이', '오사카', '지난',
#        '창춘', '텐진', '후쿠오카', '히로시마']


In [20]:
df = pd.read_csv('./data/IP2LOCATION-LITE-DB5 (1).CSV',usecols=['-.1','-.2','-.3','0.000000','0.000000.1'])
df.columns = ['country_name','city1','city2','위도','경도']
df.head()

,country_name,city1,city2,위도,경도
0,United States of America,California,Los Angeles,34.052230,-118.24368
1,China,Fujian,Fuzhou,26.061390,119.30611
2,Australia,Victoria,Melbourne,-37.814000,144.96332
3,China,Guangdong,Guangzhou,23.116670,113.25000
4,Japan,Tokyo,Tokyo,35.689506,139.69170


## 북한

In [29]:
df[df['country_name'].str.contains('Koea')]['country_name'].value_counts()

Korea (Republic of)                        59501
Korea (Democratic People's Republic of)        2
Name: country_name, dtype: int64

In [77]:
df_North = df[df['country_name']== "Korea (Democratic People's Republic of)"]
df_North.head()

#평양
lst = []
# lst.append(df_North['위도'][0])
lst.append(df_North['위도'].iloc[0])
lst.append(df_North['경도'].iloc[0])
lst

[39.03385, 125.75432]

## 일본

In [93]:
df_japan = df[df['country_name']== "Japan"]
df_japan.head()


,country_name,city1,city2,위도,경도
4,Japan,Tokyo,Tokyo,35.689506,139.69170
6,Japan,Hiroshima,Hiroshima,34.385280,132.45528
7,Japan,Miyagi,Sendai,38.267000,140.86700
8,Japan,Hiroshima,Hiroshima,34.385280,132.45528
9,Japan,Shimane,Matsue,35.467000,133.05000


In [102]:
# 영문 이름
print(df_Japan['city2'].value_counts()[:30])

# '가고시마','도쿄', '삿포로', '센다이', '오사카','후쿠오카', '히로시마','나고야'
japan_lst = ['Kagoshima','Tokyo', 'Osaka', 'Nagoya' ,'Fukuoka','Sapporo','Sendai','Funabashi']

Tokyo              47700
Osaka              10397
Yokohama            8008
Saitama             7863
Chiba               6375
Nagoya              5238
Amagasaki           3993
Fukuoka             3168
Sapporo             2834
Kyoto               2670
Sendai              1870
Shizuoka            1811
Higashimurayama     1644
Mito                1638
Maebashi            1631
Nisshin             1572
Tsukuba             1530
Marunouchi          1485
Hiroshima           1469
Utsunomiya          1384
Funabashi           1322
Okayama             1267
Niigata             1224
Otsu                1217
Kumamoto            1212
Matsumoto           1148
Fukushima           1114
Gifu                1113
Hamamatsu           1112
Kitakyushu          1062
Name: city2, dtype: int64


In [103]:
# 위도 경도 추출
df_japan.groupby('city2')[['위도','경도']].mean().loc['Tokyo']
lati = df_japan.groupby('city2')[['위도','경도']].mean().loc['Tokyo'][0]
long = df_japan.groupby('city2')[['위도','경도']].mean().loc['Tokyo'][1]
print(lati, long)

35.689506 139.6917


In [106]:
japan = []
for city in japan_lst:
    loc=[]
    df_japan.groupby('city2')[['위도','경도']].mean().loc[str(city)]
    lati = df_japan.groupby('city2')[['위도','경도']].mean().loc[str(city)][0]
    long = df_japan.groupby('city2')[['위도','경도']].mean().loc[str(city)][1]
    loc.append(lati)
    loc.append(long)
    japan.append(loc)
japan

[[31.600000000000005, 130.55],
 [35.689506, 139.6917],
 [34.69389, 135.50222],
 [35.181456, 136.906386],
 [33.583, 130.4],
 [43.067, 141.35],
 [38.267, 140.867],
 [35.7, 139.983]]

## 중국

In [107]:
df_china = df[df['country_name']== "China"]
df_china.head()

,country_name,city1,city2,위도,경도
1,China,Fujian,Fuzhou,26.06139,119.30611
3,China,Guangdong,Guangzhou,23.11667,113.25000
5,China,Guangdong,Guangzhou,23.11667,113.25000
175,China,Fujian,Fuzhou,26.06139,119.30611
177,China,Fujian,Fuzhou,26.06139,119.30611


In [111]:
# 영문이름 변경

print(df_china['city2'].value_counts()[:50])

# ['베이징','상하이', '선양','창춘', '지난']
china_lst = ['Shanghai','Beijing','Shenyang','']

Beijing         3330
Guangzhou       2143
Fuzhou          1505
Shenzhen        1187
Hangzhou        1130
Nanjing         1031
Shanghai         953
Suzhou           849
Jinan            834
Chengdu          680
Zhengzhou        627
Dongguan         514
Harbin           461
Shijiazhuang     442
Dalian           427
Qingdao          420
Foshan           407
Shenyang         399
Wuhan            396
Hefei            386
Wuxi             365
Xiamen           362
Ningbo           354
Nanchang         342
Xi'an            335
Changsha         315
Taizhou          300
Taiyuan          277
Dalianwan        272
Zhongshan        269
Jinhua           260
Haikou           260
Kunming          258
Lanzhou          256
Huizhou          245
Nantong          241
Tianjin          240
Changchun        239
Changzhou        237
Jilin            234
Shantou          233
Linyi            221
Xuzhou           217
Wenzhou          209
Nanning          208
Zhuhai           205
Yichun           201
Guiyang      

In [ ]:
df_china['']

In [65]:
import folium
map =folium.Map(location=[45,122])
folium.Marker((45,122),popup='지역이름').add_to(map)
map

In [1]:
# addr_to_lat_lon(areas[0])

In [15]:
# 날씨 측정한 곳 위도,경도 표시

# pip install folium
import folium

for area in areas:
    for i in range(len(areas)):
    
        location = addr_to_lat_lon(area[i])
        map = folium.Map(location = location, zoom_start =11)

    for a in busan_location.index:
        latitude = busan_location.loc[a,"위도"]
        longtitude = busan_location.loc[a,"경도"]
        tooltip = "사고!"
        folium.Marker([latitude, longtitude], popup = '<i>지역</i>', tooltip =tooltip).add_to(map)   

map

NameError: name 'busan_location' is not defined

In [ ]:
# 날씨 측정한 곳 위도,경도 표시

# pip install folium
import folium

for area in areas:
    
    location = addr_to_lat_lon
    map = folium.Map(location = location, zoom_start =11)

    for a in busan_location.index:
        latitude = busan_location.loc[a,"위도"]
        longtitude = busan_location.loc[a,"경도"]
        tooltip = "사고!"
        folium.Marker([latitude, longtitude], popup = '<i>지역</i>', tooltip =tooltip).add_to(map)   

map

In [ ]:
# 구제역 발생장소


In [ ]:
# 날씨 측정한 곳 위도,경도 표시


In [ ]:
# 위도 경도 거리 구하는 모듈

from haversine import haversine